In [1]:
# Importing Bot Defination
from BotDefinition import OpenAIBot
from PythonToText import get_raw_script

In [2]:
'''Choose whichever model you want to use'''
engine = "gpt-4-1106-preview"

In [3]:
chatbot = OpenAIBot(engine)

In [4]:
project_intro = r"""
This study introduces a transformative approach to Spatio-Temporal fusion in remote sensing, leveraging advanced Generative AI techniques to elevate the spatial and temporal resolutions of image sequences. Merging the powers of Long Short-Term Memory (LSTM) networks for enhancing temporal dynamics with the reconstructive power of AutoEncoders for spatial detail enhancement, the proposed neural network system processes sequential terrestrial images as an stand-in to remote sensing satellite data. Central to this work is a dataset of 16 high-resolution RAW images, each captured during sunset to faithfully reproduce the fluctuating environmental conditions characteristic of remote sensing applications. Rich in ambient light variance, these images mimic atmospheric patterns shifting over time. Processed into corresponding grey scale and RGB TIF sets, meticulous alignment paves the way for subsequent Spatio-Temporal analysis and fusion. Initiating with an LSTM-focused sub-network, the model navigates the temporal sequence of grey scale imagery, generating absent frames to refine temporal resolution. Subsequently, an autoencoder embarks on spatial extrapolation, transforming these grey scale images into full-resolution RGB translation. Beyond mere super-resolution, this process generates two complete data channels from a single grey scale channel, harnessing the capabilities of Generative AI to synthesize previously unseen information. A new composite loss function, created by integrating Mean Squared Error (MSE) with Maximum Entropy (MaxEnt) regularization, ensures a nuanced balance between reconstruction accuracy and data diversity. This loss function bridges gaps between predicted and actual data, ensuring wide-ranging applicability, better generalization and less bias.
"""

In [5]:
data = get_raw_script('Code/data.py')
autoencoder_model = get_raw_script('Code/autoencoder_model.py')
lstm_model = get_raw_script('Code/lstm_model.py')
losses = get_raw_script('Code/losses.py')
main = get_raw_script('Code/main.py')
training = get_raw_script('Code/training.py')
# evaluation = get_raw_script('Code/evaluation.py')

In [6]:
prompt = f"""

**Task: Code Analysis and Enhancement**

**Role: Machine Learning Engineer**

**Overview:**
I need assistance in refining my machine learning project. I'll provide a brief project introduction, and share existing code files. Your task is to review the code, identify issues in the LSTM module, and address missing components in the Training and Main modules.

**Project Introduction:**
{project_intro}

**Code Files:**
- **data.py:** {data}
- **autoencoder_model.py:** {autoencoder_model}
- **lstm_model.py:** {lstm_model}
- **losses.py:** {losses}
- **main.py:** {main}
- **training.py:** {training}

**Specific Concerns:**
1. **LSTM Module:** Review and suggest improvements for the LSTM module.
2. **Training Module:** Identify and fill in missing components related to LSTM training functionality.

**Assumptions:**
All necessary hardware and software resources are available.

Please analyze the code comprehensively and propose necessary changes to achieve the desired results.

"""

# evaluation.py = {evaluation}

In [7]:
# Generate and Print the Response from ChatBot
response = chatbot.generate_response(prompt)
print(response)

Let's tackle your concerns one by one, starting with the LSTM Module.

### LSTM Module Review:

**Original LSTM Model Code**
```python
class FrameInterpolationLSTM(nn.Module):
    # ...
```

Upon reviewing the LSTM module, the `FrameInterpolationLSTM` class appears to correctly implement the LSTM cell as a convolutional operation and applies it to each time step of the input image sequence for frame interpolation. However, there are couple of technical considerations to improve upon:

1. **State Initialization:** The current initialization of the hidden and cell states is not depending on the device where the model is located. It's a good practice to ensure the tensors are on the same device as the model to avoid device mismatch errors.

2. **Backward Pass Sequence:** The current LSTM implementation attempts to predict `n` intermediate frames between each pair of frames, then a weighted combination of these predictions is defined as the output for that time step, which may not be an op

In [9]:
response = chatbot.generate_response("Provide complete code for `main.py`")
print(response)

Sure, I have adjusted the `main.py` module to ensure compatibility with the provided LSTM and training enhancements previously discussed. Here's the complete code:

```python
# main.py
# Importing Custom Modules
from data import Dataset
from lstm_model import FrameInterpolationLSTM
from autoencoder_model import Grey2RGBAutoEncoder
from losses import LossMLP, LossMEP
from training import Trainer

# Import Necessary Libraries
import os
import torch
import numpy as np
from PIL import Image

# Define Working Directories
grayscale_dir = '../Dataset/Greyscale'
rgb_dir = '../Dataset/RGB'

# Define Universal Parameters
image_height = 4000
image_width = 6000
batch_size = 4
val_split = 0.2


def generate_rgb_sequence(model_lstm, model_autoencoder, grey_sequence, n_interpolate_frames, model_save_path_lstm, model_save_path_ae, generated_sequence_dir):

    if os.path.exists(model_save_path_lstm):
        state = torch.load(model_save_path_lstm)
        model_lstm.load_state_dict(state)
        mod